In [1]:
!rm -R '/content/sample_data'

In [2]:
!git clone https://github.com/RafaelKanda/IAC_project

Cloning into 'IAC_project'...
remote: Enumerating objects: 465, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 465 (delta 2), reused 9 (delta 2), pack-reused 456 (from 1)
Receiving objects: 100% (465/465), 52.71 MiB | 14.86 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [3]:
# @title Setup code
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
# import subprocess
# subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.8 MB/s eta 0:00:00
--2024-08-26 22:00:56--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2024.8.3/cloudflared-linux-amd64 [following]
--2024-08-26 22:00:56--  https://github.com/cloudflare/cloudflared/releases/download/2024.8.3/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 

In [4]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 32.0 MB/s eta 0:00:00


In [5]:
!mkdir ".streamlit"

### Criando App

In [26]:
%%writefile app.py

import streamlit as st
import pandas
from io import StringIO
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
import tempfile
from PIL import Image, ImageDraw, ImageFont, ImageOps

st.set_page_config(
    page_title="Reconhecimento de Placas Veiculares",
    page_icon="🚗",
)

st.markdown("""
    <style>
    .stApp {
        background-image: url('https://i.pinimg.com/originals/c7/0b/ea/c70bea9c667222563b35cfa5bd89d877.jpg');
        background-size: cover;
    }

    .stTitle {
        background-color: #f0f0f0;  /* Cor de fundo cinza claro */
        padding: 10px;             /* Padding para adicionar espaço ao redor do texto */
        border-radius: 5px;        /* Bordas arredondadas */
        border: 2px solid black;
        font-size: 36px;           /* Tamanho da fonte */
        color: #333;               /* Cor do texto */
        text-align: center;        /* Centralizar o texto */
        margin-bottom: 20px;       /* Espaço abaixo do título */
    }

    .stHeader {
        background-color: #f0f0f0;  /* Cor de fundo cinza claro */
        padding: 10px;             /* Padding para adicionar espaço ao redor do texto */
        border-radius: 5px;        /* Bordas arredondadas */
        border: 2px solid black;
        font-size: 22px;           /* Tamanho da fonte */
        color: #333;               /* Cor do texto */
        text-align: left;        /* Centralizar o texto */
        margin-bottom: 20px;       /* Espaço abaixo do título */
    }

    .stWrite {
        background-color: #f0f0f0;  /* Cor de fundo cinza claro */
        padding: 10px;             /* Padding para adicionar espaço ao redor do texto */
        border-radius: 5px;        /* Bordas arredondadas */
        border: 2px solid black;
        font-size: 18px;           /* Tamanho da fonte */
        color: #333;               /* Cor do texto */
        text-align: left;        /* Centralizar o texto */
        margin-bottom: 20px;       /* Espaço abaixo do título */
    }

    .stLabel {
        background-color: #f0f0f0;  /* Cor de fundo cinza claro */
        padding: 10px;             /* Padding para adicionar espaço ao redor do texto */
        border-radius: 5px;        /* Bordas arredondadas */
        border: 2px solid black;
        font-size: 16px;           /* Tamanho da fonte */
        color: #333;               /* Cor do texto */
        text-align: center;        /* Centralizar o texto */
        margin-top: 0px;
        margin-bottom: 0px;       /* Espaço abaixo do título */
    }


    </style>
""", unsafe_allow_html=True)



import cv2
import streamlit as st
from PIL import Image, ImageDraw, ImageFont, ImageOps

def draw_labeled_image(img, border_size=2, radius=5):
    """
    Desenha uma imagem com bordas arredondadas, uma borda preta arredondada e uma label centralizada abaixo da imagem.

    Parâmetros:
        image_path (str): O caminho para a imagem.
        label (str): O texto da label a ser exibida abaixo da imagem.
        border_size (int): O tamanho da borda preta ao redor da imagem. (Padrão: 10)
        radius (int): O raio das bordas arredondadas. (Padrão: 30)
        label_height (int): A altura do espaço para a label abaixo da imagem. (Padrão: 40)
        font_size (int): O tamanho da fonte do texto da label. (Padrão: 20)

    Retorna:
        Image: A imagem final com bordas arredondadas, borda preta arredondada e label.
    """

    # Converter para PIL para manipulação
    pil_img = Image.fromarray(img)

    # Criar uma máscara com cantos arredondados
    mask = Image.new('L', pil_img.size, 0)
    draw = ImageDraw.Draw(mask)
    draw.rounded_rectangle(
        (0, 0, pil_img.size[0], pil_img.size[1]),
        radius=radius,  # Ajuste o raio conforme necessário
        fill=255
    )

    # Aplicar máscara para obter a imagem com cantos arredondados
    rounded_img = ImageOps.fit(pil_img, mask.size, centering=(0.5, 0.5))
    rounded_img.putalpha(mask)

    # Criar nova imagem com a borda arredondada
    bordered_img = Image.new('RGBA', (rounded_img.width + 2 * border_size, rounded_img.height + 2 * border_size))
    border_mask = Image.new('L', bordered_img.size, 0)
    draw = ImageDraw.Draw(border_mask)
    draw.rounded_rectangle(
        (0, 0, bordered_img.size[0], bordered_img.size[1]),
        radius=radius + border_size,  # Aumentar o raio para incluir a borda
        fill=255
    )

    # Colocar a imagem arredondada no centro da nova imagem com borda
    bordered_img.paste((0, 0, 0, 255), (0, 0), border_mask)
    bordered_img.paste(rounded_img, (border_size, border_size), rounded_img)

    return bordered_img



def extract_plate(img): # the function detects and perfors blurring on the number plate.
	plate_img = img.copy()

	#Loads the data required for detecting the license plates from cascade classifier.
	plate_cascade = cv2.CascadeClassifier('/content/IAC_project/indian_license_plate.xml')

	# detects numberplates and returns the coordinates and dimensions of detected license plate's contours.
	plate_rect = plate_cascade.detectMultiScale(plate_img, scaleFactor = 1.3, minNeighbors = 7)

	for (x,y,w,h) in plate_rect:
		a,b = (int(0.01*img.shape[0]), int(0.015*img.shape[1])) #parameter tuning
		plate = plate_img[y+a:y+h-a, x+b:x+w-b, :]


		# finally representing the detected contours by drawing rectangles around the edges.
		cv2.rectangle(plate_img, (x,y), (x+w, y+h), (51,51,255), 3)

	return plate_img, plate # returning the processed image.



def find_contours(dimensions, img):
    # Pré-processamento da imagem
    if len(img.shape) == 3:  # Se a imagem tem 3 canais, converta para escala de cinza
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:  # Se já estiver em escala de cinza, continue com a imagem original
        gray_img = img
    _, binary_img = cv2.threshold(gray_img, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Operação morfológica de fechamento para preencher buracos pequenos
    kernel = np.ones((3,3), np.uint8)
    closed_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)

    # Visualize a imagem processada
    final_closed_img = draw_labeled_image(closed_img, radius=4, border_size= 2)
    st.markdown('<p class="stLabel">Pré-Processada</p>', unsafe_allow_html=True)
    st.image(final_closed_img, width=700)

    # Encontre todos os contornos na imagem, focando apenas nos contornos externos
    cntrs, hierarchy = cv2.findContours(closed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Defina as dimensões mínimas e máximas para largura e altura
    lower_width, upper_width, lower_height, upper_height = dimensions

    # Selecione os maiores 15 contornos (ou os 5 maiores, conforme necessário)
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]

    # Crie uma imagem para desenhar os contornos
    img_res = np.zeros_like(img)

    # Listas para armazenar coordenadas x e imagens de caracteres
    x_cntr_list = []
    img_res_list = []

    for cntr in cntrs:
        # Encontre o retângulo delimitador ao redor do contorno
        x, y, w, h = cv2.boundingRect(cntr)

        # Filtre contornos com base nas dimensões fornecidas
        if lower_width < w < upper_width and lower_height < h < upper_height:
            x_cntr_list.append(x)

            # Extraia e redimensione o caractere
            char = closed_img[y:y+h, x:x+w]
            char = cv2.resize(char, (20, 40))

            # Inverter as cores do caractere
            char = cv2.bitwise_not(char)

            # Desenhe o retângulo delimitador na imagem
            cv2.rectangle(img_res, (x, y), (x + w, y + h), (50, 21, 200), 2)

            # Prepare a imagem do caractere
            char_copy = np.zeros((44, 24), dtype=np.uint8)
            char = cv2.subtract(255, char)
            char_copy[2:42, 2:22] = char

            # Adicione bordas pretas
            char_copy[0:2, :] = 0
            char_copy[:, 0:2] = 0
            char_copy[42:44, :] = 0
            char_copy[:, 22:24] = 0

            img_res_list.append(char_copy)

    # Exiba a imagem com contornos detectados
    final_img_res = draw_labeled_image(img_res, radius=4, border_size= 2)
    st.markdown('<p class="stLabel">Contornos</p>', unsafe_allow_html=True)
    st.image(final_img_res, width=700)

    # Classifique os caracteres com base na coordenada x
    sorted_indices = sorted(range(len(x_cntr_list)), key=lambda k: x_cntr_list[k])
    img_res_list = [img_res_list[idx] for idx in sorted_indices]

    # Determina o número de imagens
    num_images = len(img_res_list)

    # Cria uma coluna para cada imagem
    cols = st.columns(num_images)


    # Converta para array NumPy e retorne
    img_res_array = np.array(img_res_list)

    return img_res_array





# Separar os caracteres
def segment_characters(image) :

    # Preprocess cropped license plate image
    img_lp = cv2.resize(image, (333, 75))
    img_gray_lp = cv2.cvtColor(img_lp, cv2.COLOR_BGR2GRAY)
    #_, img_binary_lp = cv2.threshold(img_gray_lp, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    _, img_binary_lp = cv2.threshold(img_gray_lp, 150, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_binary_lp = cv2.erode(img_binary_lp, (3,3))
    img_binary_lp = cv2.dilate(img_binary_lp, (3,3))

    LP_WIDTH = img_binary_lp.shape[0]
    LP_HEIGHT = img_binary_lp.shape[1]

    # Make borders white
    img_binary_lp[0:3,:] = 255
    img_binary_lp[:,0:3] = 255
    img_binary_lp[72:75,:] = 255
    img_binary_lp[:,330:333] = 255

    dimensions = [LP_WIDTH/10, LP_WIDTH, LP_HEIGHT/20, LP_HEIGHT]
    #dimensions = [LP_WIDTH/8, LP_WIDTH, LP_HEIGHT/15, LP_HEIGHT]



    plt.imshow(img_binary_lp, cmap='gray')
    plt.show()

    cv2.imwrite('contour.jpg',img_binary_lp)

    # Get contours within cropped license plate
    char_list = find_contours(dimensions, img_binary_lp)

    return char_list

# Função auxiliar de corrigir dimensão
def fix_dimension(img):
    new_img = np.zeros((28,28,3))
    for i in range(3):
        new_img[:,:,i] = img
    return new_img

# Função para pegar os resultados
def show_results(char):
    dic = {}
    characters = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i,c in enumerate(characters):
        dic[i] = c

    output = []
    for i,ch in enumerate(char): #iterating over the characters
        img_ = cv2.resize(ch, (28,28))
        img = fix_dimension(img_)
        img = img.reshape(1,28,28,3) #preparing image for the model
        y_ = model_ada.predict(img)[0] #predicting the class
        y_int = np.argmax(y_)
        character = dic[y_int] #
        output.append(character) #storing the result in a list

    plate_number = ''.join(output)

    return plate_number


# Treinando modelo da CNN
model_ada = tf.keras.models.load_model('/content/IAC_project/model_adamax.keras')

# Título da página
st.markdown('<h1 class="stTitle">Reconhecimento de Placas Veiculares</h1>', unsafe_allow_html=True)
st.markdown('<p class="stWrite">Insira a imagem do carro:</p>', unsafe_allow_html=True)

# Upload da imagem
uploaded_file = st.file_uploader("File upload", type=['png', 'jpg'])

# Criando caminho para a imagem
if uploaded_file is not None:
    st.markdown('<h1 class="stHeader">Processamento:</h1>', unsafe_allow_html=True)
    # Criação de um diretório temporário
    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
        # Salvar a imagem carregada no arquivo temporário
        temp_file.write(uploaded_file.read())
        caminho_imagem = temp_file.name

    # Mostrando a imagem recebida
    img = cv2.imread(caminho_imagem)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    final_img = draw_labeled_image(img)
    st.markdown('<p class="stLabel">Original</p>', unsafe_allow_html=True)
    st.image(final_img, width=700)



    # Detectando e extraindo a placa
    try:
      img_dtc, plate = extract_plate(img)
      # Mostrando a imagem com detecção
      final_img_dtc = draw_labeled_image(img_dtc)
      st.markdown('<p class="stLabel">Detecção</p>', unsafe_allow_html=True)
      st.image(final_img_dtc, width=700)

      # Mostrando a placa
      final_plate = draw_labeled_image(plate, radius=10, border_size= 2)
      st.markdown('<p class="stLabel">Placa Detectada</p>', unsafe_allow_html=True)
      st.image(final_plate, width=700)

      # Segmentando a placa
      char = segment_characters(plate)

      # Mostrando os caracteres
      string_placa = show_results(char)

      if string_placa is not None:
        st.markdown(f'<h3 class="stHeader">Placa detectada: {string_placa}</h3>', unsafe_allow_html=True)
      else:
        st.markdown('<h3>Sinto muito! Não foi possível identificar os caracteres.</h3>', unsafe_allow_html=True)
    except:
      st.markdown('<h3>Sinto muito! Não foi possível detectar a placa.</h3>', unsafe_allow_html=True)



Overwriting app.py


In [7]:
!streamlit run /content/app.py &>/content/logs.txt &

In [8]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://rim-eau-operates-sponsorship.trycloudflare.com
